# mts

> mts ist ein Web Tool, welches Rechnungen aus einem bestehenden CRM/ERP in das im Schweizer Gesundheitswesen verwendete GeneralInvoice-XML-Format umwandelt und via MediData an die Invalidenversicherung (IV) weiterreicht. Daten-Empfänger für alle IV-Stellen der Schweiz ist die Zentrale Ausgleichsstelle ZAS. 


In [ ]:
#| default_exp matlon_mts

In [ ]:
#| hide
#| echo: false
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *
from IPython.display import Markdown
from matlon.model import *


(Because this is a tool specifically for the Swiss healthcare system, this page is in German instead of English.)

## Wozu mts?
Zitat von der [ZAS-Website](https://www.zas.admin.ch/zas/de/home/partenaires-et-institutions-/paiement-des-prestations-individuelles-avs-ai/information-a-l-attention-des-fournisseurs-de-prestations.html):

> "Die ZAS strebt langfristig eine fast ausschliesslich elektronische Bearbeitung von Rechnungen für individuelle AHV/IV-Leistungen an."

Leistungserbringer, deren Software nicht in der Lage ist, Rechnungen in das GeneralInvoice-XML-Format zu exportieren und zu übermitteln, stellt dies vor grosse Herausforderungen. Sie müssen entweder auf eine neue Software umsteigen, oder die Rechnungen nochmals mit allen Details manuell in einem Web-Formular erfassen.

Mit __mts__ bietet matlon ein schlankes Tool an, welches alle notwendigen Rechnungsdaten aus dem vorhandenen CRM/ERP exportiert, validiert und in das GeneralInvoice-XML-Format umwandelt. mts orchestriert die anschliessende Übermittlung dieser Daten via MediData-Netz an die ZAS und die Verarbeitung und Quittierung allfälliger Rückweisungs-Meldungen der ZAS / IV-Stellen.

Screenshot:
![mts Dashboard](mts1.jpg)

Die GeneralInvoice TP Rechnungen können auch als PDF heruntergeladen werden. Für Rechnungen aus privaten Aufträgen (die nicht über die IV abgerechnet werden) kann mts auch QR-Einzahlungsscheine generieren.

## Welche CRM/ERP-Systeme werden unterstützt?
Wir programmieren im Rahmen eines Projekts einen Adapter, der exakt zu Ihrem vorhandenen System und den dortigen Datenstrukturen passt.
Für [b-at.ch](https://www.b-at.ch) haben wir beispielsweise einen Adapter geschrieben, der die Daten aus der [Teamleader](https://www.teamleader.eu)-Software exportiert.


# Diagramme
## Sequenzdiagramm
```{mermaid}
sequenceDiagram
    actor User as Benutzer
    participant CRM as CRM<br>z.B. Teamleader
    participant mts as mts<br>CRM Daten zu generalInvoice XML Format<br>Daten-Konverter Plattform für Leistungsabrechnungen
    participant MediData
    participant D as Empfänger<br>z.B. IV/ZAS
    User->>CRM: Rechnung erstellen
    mts->>+CRM: neue Rechnungen abholen
    mts->>+mts: Rechnungen validieren und als XML aufbereiten
    activate mts
    User->>mts: Prüfen und Freigabe
    mts->>MediData: Rechnung übermitteln
    mts->>CRM: Status-Notiz erstellen
    deactivate mts
    MediData->>+D: Rechnung weiterreichen
    D->>D: Rechnung prüfen, falls abgewiesen: Nachricht
    D->>+MediData: Resultat zurückmelden
    activate mts
    mts->>+MediData: Resultate und Nachrichten abholen
    mts->>+CRM: Status-Notiz erstellen
    deactivate mts
    User->>mts: Nachrichten ansehen und Quittieren
```
## Statusdiagramm
```{mermaid}
stateDiagram-v2
    [*] --> ignoriert: Ignorieren Aktion
    [*] --> registriert: Sync Aktion
    registriert --> validiert: Validierung erfolgreich
    registriert --> Validierung_fehlgeschlagen: Validierungsfehler
    Validierung_fehlgeschlagen --> validiert: Sync Aktion
    Validierung_fehlgeschlagen --> ignoriert: Ignorieren Aktion
    validiert --> exportiert_zu_MediData: Zu MediData hochladen Aktion
    validiert --> validiert: Sync Aktion
    validiert --> ignoriert: Ignorieren Aktion
    validiert --> Export_zu_MediData_fehlgeschlagen: Zu MediData hochladen Aktion, MediData nicht erreichbar
    exportiert_zu_MediData --> akzeptiert: Zahlungseingang
    exportiert_zu_MediData --> zurückgewiesen: Antwort oder Notifikation
    Export_zu_MediData_fehlgeschlagen --> exportiert_zu_MediData: Zu MediData hochladen Aktion
    zurückgewiesen --> [*]: Nachdem Antwort oder Notifikation bestätigt wurde
    akzeptiert --> [*]
    ignoriert --> [*]
    
    note right of registriert: temporärer Übergangsstatus
    
    classDef transient stroke-dasharray: 5 5, fill: white
    class registriert transient
```


# Interessiert?
Kontaktieren Sie [Andri Krämer](mailto:akraemer@matlon.ch).


# Nützliche Links
- [ZAS Infos zur elektronischen Rechnungsstellung AHV/IV](https://www.zas.admin.ch/zas/de/home/partenaires-et-institutions-/paiement-des-prestations-individuelles-avs-ai/information-a-l-attention-des-fournisseurs-de-prestations/fournisseur_de_prestations.html)
- [Forum Datenaustausch: Elektronische Kommunikation im Gesundheitswesen](https://www.forum-datenaustausch.ch/de/)
- [MediData](https://www.medidata.ch/md/web/de/)
- [b-at AG Beratung assistive Technologien](https://www.b-at.ch) (Referenzkunde)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()